In [1]:
import pandas as pd
import collections

with open('quran-text/quran-uthmani.txt') as f:
    quran_text = [l.strip() for l in f if not l.strip().startswith('#') and l.strip()]

In [2]:
FILTERED_CHARS = \
    '|123456789 \t\nًٌَُّْ	' \
    'ِ ۫ ۨ ۬ ۬' \
    '۪ ۣۭۜ۠' \
    '	' \
    'ٰ'
def filter_char(text):
    return ''.join(c for c in text if c not in FILTERED_CHARS)

# split basmallah to its own
i = 0 
while i < len(quran_text):
    aya = quran_text[i].split('|')
    if aya[2].startswith('بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ '):
        quran_text.insert(i, f'{aya[0]}|0|بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ')
        i += 1
        quran_text[i] = f'{aya[0]}|1|{aya[2][38:]}'
    i += 1

df = pd.DataFrame({'sura': int(a[0]), 'verse': int(a[1]), 'text': a[2]} for a in [a.split('|') for a in quran_text])
df

,sura,verse,text
0,1,1,بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ
1,1,2,ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَٰلَمِينَ
2,1,3,ٱلرَّحْمَٰنِ ٱلرَّحِيمِ
3,1,4,مَٰلِكِ يَوْمِ ٱلدِّينِ
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...
6341,114,2,مَلِكِ ٱلنَّاسِ
6342,114,3,إِلَٰهِ ٱلنَّاسِ
6343,114,4,مِن شَرِّ ٱلْوَسْوَاسِ ٱلْخَنَّاسِ
6344,114,5,ٱلَّذِى يُوَسْوِسُ فِى صُدُورِ ٱلنَّاسِ


In [3]:
# NOTE: this count و as integral part of a word
df['words'] = df.text.apply(lambda t: [w for w in [filter_char(w) for w in t.split(' ')] if w.strip()])
df['word_count'] = df.words.apply(lambda w: len(w))

In [4]:
df.head(10)

,sura,verse,text,words,word_count
0,1,1,بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ,"[بسم, ٱلله, ٱلرحمن, ٱلرحيم]",4
1,1,2,ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَٰلَمِينَ,"[ٱلحمد, لله, رب, ٱلعلمين]",4
2,1,3,ٱلرَّحْمَٰنِ ٱلرَّحِيمِ,"[ٱلرحمن, ٱلرحيم]",2
3,1,4,مَٰلِكِ يَوْمِ ٱلدِّينِ,"[ملك, يوم, ٱلدين]",3
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"[إياك, نعبد, وإياك, نستعين]",4
5,1,6,ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ,"[ٱهدنا, ٱلصرط, ٱلمستقيم]",3
6,1,7,صِرَٰطَ ٱلَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...,"[صرط, ٱلذين, أنعمت, عليهم, غير, ٱلمغضوب, عليهم...",9
7,2,0,بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ,"[بسم, ٱلله, ٱلرحمن, ٱلرحيم]",4
8,2,1,الٓمٓ,[الٓمٓ],1
9,2,2,ذَٰلِكَ ٱلْكِتَٰبُ لَا رَيْبَ فِيهِ هُدًى لِّل...,"[ذلك, ٱلكتب, لا, ريب, فيه, هدى, للمتقين]",7


In [5]:
# longest verse by word count
df.sort_values('word_count', ascending=False).head(10)

,sura,verse,text,words,word_count
289,2,282,يَٰٓأَيُّهَا ٱلَّذِينَ ءَامَنُوٓا۟ إِذَا تَدَا...,"[يٓأيها, ٱلذين, ءامنوٓا۟, إذا, تداينتم, بدينٍ,...",128
507,4,12,وَلَكُمْ نِصْفُ مَا تَرَكَ أَزْوَٰجُكُمْ إِن ل...,"[ولكم, نصف, ما, ترك, أزوجكم, إن, لم, يكن, لهن,...",88
5565,73,20,إِنَّ رَبَّكَ يَعْلَمُ أَنَّكَ تَقُومُ أَدْنَى...,"[إن, ربك, يعلم, أنك, تقوم, أدنى, من, ثلثى, ٱلي...",78
2843,24,31,وَقُل لِّلْمُؤْمِنَٰتِ يَغْضُضْنَ مِنْ أَبْصَٰ...,"[وقل, للمؤمنت, يغضضن, من, أبصرهن, ويحفظن, فروج...",78
2873,24,61,لَّيْسَ عَلَى ٱلْأَعْمَىٰ حَرَجٌ وَلَا عَلَى ٱ...,"[ليس, على, ٱلأعمى, حرج, ولا, على, ٱلأعرج, حرج,...",76
448,3,154,ثُمَّ أَنزَلَ عَلَيْكُم مِّنۢ بَعْدِ ٱلْغَمِّ ...,"[ثم, أنزل, عليكم, منۢ, بعد, ٱلغم, أمنة, نعاسا,...",75
109,2,102,وَٱتَّبَعُوا۟ مَا تَتْلُوا۟ ٱلشَّيَٰطِينُ عَلَ...,"[وٱتبعوا۟, ما, تتلوا۟, ٱلشيطين, على, ملك, سليم...",74
203,2,196,وَأَتِمُّوا۟ ٱلْحَجَّ وَٱلْعُمْرَةَ لِلَّهِ فَ...,"[وأتموا۟, ٱلحج, وٱلعمرة, لله, فإن, أحصرتم, فما...",73
506,4,11,يُوصِيكُمُ ٱللَّهُ فِىٓ أَوْلَٰدِكُمْ لِلذَّكَ...,"[يوصيكم, ٱلله, فىٓ, أولدكم, للذكر, مثل, حظ, ٱل...",71
3616,33,53,يَٰٓأَيُّهَا ٱلَّذِينَ ءَامَنُوا۟ لَا تَدْخُلُ...,"[يٓأيها, ٱلذين, ءامنوا۟, لا, تدخلوا۟, بيوت, ٱل...",69


In [6]:
# shortest verse by word count
df.sort_values('word_count').head(10)

,sura,verse,text,words,word_count
2366,20,1,طه,[طه],1
5390,69,1,ٱلْحَآقَّةُ,[ٱلحآقة],1
6080,89,1,وَٱلْفَجْرِ,[وٱلفجر],1
3739,36,1,يسٓ,[يسٓ],1
4554,46,1,حمٓ,[حمٓ],1
4516,45,1,حمٓ,[حمٓ],1
5017,55,64,مُدْهَآمَّتَانِ,[مدهآمتان],1
6275,103,1,وَٱلْعَصْرِ,[وٱلعصر],1
960,7,1,الٓمٓصٓ,[الٓمٓصٓ],1
2267,19,1,كٓهيعٓصٓ,[كٓهيعٓصٓ],1


In [21]:
df_words = df[['sura','verse','words']].explode('words').rename(columns={'words': 'word'})
df_words

,sura,verse,word
0,1,1,بسم
0,1,1,ٱلله
0,1,1,ٱلرحمن
0,1,1,ٱلرحيم
1,1,2,ٱلحمد
...,...,...,...
6344,114,5,صدور
6344,114,5,ٱلناس
6345,114,6,من
6345,114,6,ٱلجنة


In [22]:
#longest word in the quran
df_words['wordlen'] = df_words['word'].apply(len)
df_words.sort_values('wordlen', ascending=False).head(10)

,sura,verse,word,wordlen
1424,10,53,ويستنۢبـٔونك,12
2871,24,59,فليستـٔذنوا۟,12
1838,15,24,ٱلمستـٔخرين,11
622,4,127,وٱلمستضعفين,11
4262,41,6,فٱستقيموٓا۟,11
3197,27,14,وٱستيقنتهآ,11
570,4,75,وٱلمستضعفين,11
578,4,83,يستنۢبطونهۥ,11
4491,44,36,بـٔابآئنآ,11
2174,18,19,ليتسآءلوا۟,11


In [30]:
df_word_occurence = df_words[['word','verse']].groupby('word').count().rename(columns={'verse': 'count'}).reset_index()
# most occuring word
df_word_occurence.sort_values('count', ascending=False).head(10)

,word,count
9373,من,2589
15444,ٱلله,2265
6910,فى,1099
1447,إن,966
8709,ما,873
14998,ٱلذين,811
7850,لا,749
1091,أن,637
11637,ولا,605
5734,على,605


In [37]:
# word that has the same occurences
df_word_same_occurence = df_word_occurence.groupby('count').agg(list)
df_word_same_occurence = df_word_same_occurence[df_word_same_occurence['word'].apply(len) > 1]
df_word_same_occurence.sort_index(ascending=False).head(20)

,word
count,
605,"[على, ولا]"
337,"[لكم, ومن]"
280,"[أو, ذلك]"
182,"[ٱلسموت, ٱلناس]"
164,"[عليكم, وٱلذين]"
157,"[وٱلأرض, ٱلرحمن]"
146,"[عليه, ٱلرحيم]"
137,"[عذاب, مآ, وهم]"
127,"[بل, رب]"
